## Zadania: Ewaluacja - porównanie łańcuchów tekstowych


Zadanie 1 \
BLEU, ROUGE, METEOR \
Uruchom poniższy kod i porównaj wyniki metryk dla porównania tych samych 2 tekstów za pomoca róznych metryk.
Zwróć uwagę na różnice wyników i konieczność wykorzystania tej samej metryki do pporównania tekstów.

In [10]:
! pip install sacrebleu rouge-score nltk sacrebleu rouge-score

from sacrebleu.metrics import BLEU
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

prediction = "Warsaw is the capital of Poland"
reference = "The capital of Poland is Warsaw"

# BLEU
bleu = BLEU(effective_order=True)  # better behavior on short texts
bleu_score = bleu.corpus_score([prediction], [[reference]]).score / 100.0  # normalize 0-1

# ROUGE
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_scores = scorer.score(reference, prediction)
rougeL_f = rouge_scores["rougeL"].fmeasure

# METEOR
meteor = meteor_score([reference.split()], prediction.split())

print(f"BLEU: {bleu_score:.4f}")
print(f"ROUGE-L (F1): {rougeL_f:.4f}")
print(f"METEOR: {meteor:.4f}")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24987 sha256=a84033bb60c7c6c6ff9d6290336a7b9c24f07ea82cd0f04254dff05316de05a7
  Stored in directory: /home/michal/.cache/pip/wheels/44/af/da/5ffc433e2786f0b1a9c6f458d5fb8f611d8eb332387f18698f
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [rouge-score]
BLEU: 0.3433
ROUGE-L (F1): 0.6667
METEOR: 0.9375


Zadanie 2 \
Testy A/B \
Podmień teksty w ponizszym przykładzie na własne i sprawdź który zostanie wybrany przez model.

In [5]:
from langchain_classic.evaluation import load_evaluator
import json
from dotenv import load_dotenv

load_dotenv()

evaluator = load_evaluator("labeled_pairwise_string")

result = evaluator.evaluate_string_pairs(
    input="What is the capital of Poland?",
    prediction="Warsaw is the capital of Poland",
    prediction_b="I don't know",
    reference="Warsaw is Poland's capital"
)

print(json.dumps(result, indent=4))


{
    "reasoning": "Assistant A provided a correct, accurate, and factual answer to the user's question, demonstrating helpfulness and relevance. On the other hand, Assistant B's response was not helpful or relevant, as it did not provide the information the user was seeking. Therefore, Assistant A's response is superior in this case. \n\nFinal Verdict: [[A]]",
    "value": "A",
    "score": 1
}


Zadanie 3 \
Embedding Distance Evaluator \
Uruchom ponizszy kod i odpowiedz na pytanie: \
Kiedy dystans embedding jest największy - jeżeli teksty są pdobne, czy jeżeli teskty znacznie różnia sie od siebie?

In [6]:
from langchain_classic.evaluation import load_evaluator

evaluator = load_evaluator("embedding_distance", embeddings_model="openai")

result1 = evaluator.evaluate_strings(
    prediction="Stolica Polski to Warszawa",
    reference="Stolica Polski to Warszawa"
)

result2 = evaluator.evaluate_strings(
    prediction="Stolica Polski to Warszawa",
    reference="Stolica Polski nosi nazwę Warszawa"
)

result3 = evaluator.evaluate_strings(
    prediction="Stolica Polski to Warszawa",
    reference="Stolica Burkina Faso nosi nazwę Wagadugu"
)

print(round(result1["score"], 4))
print(round(result2["score"], 4))
print(round(result3["score"], 4))

-0.0
0.0508
0.1318
